In [160]:
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import spmatrix

In [1]:
adata = sc.read_h5ad("merscope_855_notscaled.h5ad")
print(adata)

NameError: name 'sc' is not defined

In [180]:
adata_scaled = sc.read_h5ad("merscope_855.h5ad")
print(adata_scaled)

AnnData object with n_obs × n_vars = 15927370 × 300
    obs: 'sample', 'H1_annotation', 'H2_annotation', 'H3_annotation'
    obsm: 'X_pca', 'X_spatial'


In [181]:
adata.obs.index = adata.obs.index.map(lambda x: x.split('-')[0])
adata.obs['H3_annotation'] = adata_scaled.obs['H3_annotation']
print(adata.obs['H3_annotation'].head())

2241155200012100016      EN-IT-L3-V1
2241155200012100018      EN-IT-L3-V1
2241155200012100021          EN-IZ-3
2241155200012100028    EN-IT-L2/3-A2
2241155200012100030       EN-IT-L3-A
Name: H3_annotation, dtype: category
Categories (114, object): ['Astro-1', 'Astro-2', 'Astro-early', 'Astro-late1', ..., 'vRG-early2', 'vRG-late1', 'vRG-late2', 'vRG-late3']


In [262]:
clusters_of_interest = ['EN-ET-SP-A', 'EN-ET-SP-P1']
#clusters_of_interest = ['EN-ET-SP-A', 'EN-ET-SP-P1']
adata_subset = adata[adata.obs['H3_annotation'] == clusters_of_interest[0]].copy()
adata_subset2 = adata[adata.obs['H3_annotation']== clusters_of_interest[1]].copy()

/Users/cyluo/anaconda3/envs/xuyuEnv/lib/python3.9/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/cyluo/anaconda3/envs/xuyuEnv/lib/python3.9/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [263]:
print(adata_subset)

AnnData object with n_obs × n_vars = 91381 × 300
    obs: 'n_counts', 'sample', 'region', 'gw', 'H3_annotation'
    uns: 'rank_genes_groups'
    obsm: 'spatial'


In [264]:
dense_X = adata_subset.X.toarray()
mean_expression = np.mean(dense_X, axis=0)

dense_X2 = adata_subset2.X.toarray()
mean_expression2 = np.mean(dense_X2, axis=0)

In [265]:
expression_threshold = 0  
expressed_genes_mask = dense_X > expression_threshold
num_cells_expressing = np.sum(expressed_genes_mask, axis=0)

expressed_genes_mask2 = dense_X2 > expression_threshold
num_cells_expressing2 = np.sum(expressed_genes_mask2, axis=0)

genes_expressed_per_cell = np.sum(expressed_genes_mask, axis=1)
total_expressed_cells = np.sum(genes_expressed_per_cell > 0)
percent_expressed = num_cells_expressing / total_expressed_cells * 100

genes_expressed_per_cell2 = np.sum(expressed_genes_mask2, axis=1)
total_expressed_cells2 = np.sum(genes_expressed_per_cell2 > 0)
percent_expressed2 = num_cells_expressing2 / total_expressed_cells2 * 100

In [266]:
gene_names = adata_subset.var_names
mean_expression_per_gene = pd.Series(mean_expression, index=gene_names)
percent_expressed_per_gene = pd.Series(percent_expressed, index=gene_names)
percent_expressed_per_gene = percent_expressed_per_gene.to_frame(name='expressed cells (%)')

gene_names2 = adata_subset2.var_names
mean_expression_per_gene2 = pd.Series(mean_expression2, index=gene_names)
percent_expressed_per_gene2 = pd.Series(percent_expressed2, index=gene_names)
percent_expressed_per_gene2 = percent_expressed_per_gene2.to_frame(name='expressed cells (%)')

In [267]:
print(mean_expression_per_gene)
print(percent_expressed_per_gene)

HS3ST1      0.023180
CD9         0.009612
CBLN4       0.028334
CA12        0.056644
LGALS1      0.009583
              ...   
SORCS1      0.337071
ANGPT2      0.012900
PAX6        0.016420
POU3F4      0.009900
NIPBL-DT    0.199339
Length: 300, dtype: float32
          expressed cells (%)
HS3ST1               5.456276
CD9                  2.266335
CBLN4                6.213545
CA12                12.737878
LGALS1               2.246638
...                       ...
SORCS1              47.616025
ANGPT2               3.112244
PAX6                 3.873891
POU3F4               2.248826
NIPBL-DT            37.877677

[300 rows x 1 columns]


In [286]:
de_df = pd.read_csv('gene_output5.csv')
# Sort by log fold change
sorted_de_df = de_df.sort_values(by=['log_fold_change', '-log10_pval'], ascending=[False, False])
# Select top 10/20 and bottom 10/20 genes
N = 20
top_genes = sorted_de_df.head(N)
bottom_genes = sorted_de_df.tail(N)
# Combine top and bottom genes
selected_genes = pd.concat([top_genes, bottom_genes])
print(selected_genes)

     Unnamed: 0       gene  log_fold_change  pvals  pvals_adj  -log10_pval
0             0     HCRTR2         5.094426    0.0        0.0          inf
1             1      NR4A2         4.886286    0.0        0.0          inf
2             2       MLIP         2.690638    0.0        0.0          inf
3             3    CYP26A1         2.426295    0.0        0.0          inf
6             6       ST18         2.421855    0.0        0.0          inf
10           10     STK32B         2.420835    0.0        0.0          inf
12           12      UNC5C         2.203939    0.0        0.0          inf
21           21    SLC26A4         2.147651    0.0        0.0          inf
4             4     ADRA2A         2.114066    0.0        0.0          inf
7             7       NPR3         1.720079    0.0        0.0          inf
5             5      SATB2         1.553550    0.0        0.0          inf
8             8     GABRA5         1.533800    0.0        0.0          inf
15           15       CCN

In [287]:
# Assuming mean_expression_per_gene is a pandas Series and selected_genes is a pandas DataFrame
selected_gene_names = selected_genes['gene']
filtered_mean_expression = mean_expression_per_gene[mean_expression_per_gene.index.isin(selected_gene_names)]
filtered_mean_expression2 = mean_expression_per_gene2[mean_expression_per_gene2.index.isin(selected_gene_names)]
filtered_percent_expressed_per_gene = percent_expressed_per_gene[percent_expressed_per_gene.index.isin(selected_gene_names)]
filtered_percent_expressed_per_gene2 = percent_expressed_per_gene2[percent_expressed_per_gene2.index.isin(selected_gene_names)]

In [288]:
df_mean_expression = filtered_mean_expression.to_frame(name='mean expression')
print(df_mean_expression.head())

df_mean_expression2 = filtered_mean_expression2.to_frame(name='mean expression')
print(df_mean_expression2.head())

         mean expression
HS3ST1          0.023180
GLRA2           0.323478
CYP26A1         0.673407
LGR6            0.008685
ST18            0.335150
         mean expression
HS3ST1          0.063015
GLRA2           0.181025
CYP26A1         0.164469
LGR6            0.030985
ST18            0.071672


In [289]:
combined_df_cluster1 = df_mean_expression.join(filtered_percent_expressed_per_gene, how='inner')
combined_df_cluster2 = df_mean_expression2.join(filtered_percent_expressed_per_gene2, how='inner')
print(combined_df_cluster1)

           mean expression  expressed cells (%)
HS3ST1            0.023180             5.456276
GLRA2             0.323478            50.916492
CYP26A1           0.673407            76.546547
LGR6              0.008685             2.060603
ST18              0.335150            51.879494
ADRA2A            0.666940            74.267080
STK32B            0.115034            23.180968
ADAMTSL3          0.023853             5.564614
VAT1L             0.086918            17.228964
KCNH5             0.016095             3.779779
NPR3              0.318118            48.576838
NR2F1             0.216569            39.175540
NR4A2             1.420112            95.502347
ELOVL2            0.077923            16.599731
KIF26B            0.082806            15.604994
CCN2              0.184081            29.275232
KCNQ5             0.073841            15.954082
HCRTR2            0.719916            99.348880
PCDH17            0.367110            52.571103
LRP4              0.021096             4

In [290]:
combined_df_cluster1['cluster'] = clusters_of_interest[0]
combined_df_cluster2['cluster'] = clusters_of_interest[1]

In [291]:
print(combined_df_cluster2.head)

<bound method NDFrame.head of            mean expression  expressed cells (%)      cluster
HS3ST1            0.063015            14.614021  EN-ET-SP-P1
GLRA2             0.181025            34.614836  EN-ET-SP-P1
CYP26A1           0.164469            32.805656  EN-ET-SP-P1
LGR6              0.030985             7.629933  EN-ET-SP-P1
ST18              0.071672            16.162419  EN-ET-SP-P1
ADRA2A            0.198067            32.884094  EN-ET-SP-P1
STK32B            0.022511             5.494774  EN-ET-SP-P1
ADAMTSL3          0.063232            15.081596  EN-ET-SP-P1
VAT1L             0.311741            51.686938  EN-ET-SP-P1
KCNH5             0.209332            37.247112  EN-ET-SP-P1
NPR3              0.107674            23.184198  EN-ET-SP-P1
NR2F1             0.100981            22.383514  EN-ET-SP-P1
NR4A2             0.100834            17.916590  EN-ET-SP-P1
ELOVL2            0.232318            44.741560  EN-ET-SP-P1
KIF26B            0.213378            39.010452  EN-ET-

In [283]:
# Ensure the gene names are aligned and sorted
combined_df_cluster1 = combined_df_cluster1.sort_index()
combined_df_cluster2 = combined_df_cluster2.sort_index()

In [292]:
combined_df_cluster1 = combined_df_cluster1.loc[selected_genes['gene']]
combined_df_cluster2 = combined_df_cluster2.loc[selected_genes['gene']]
print(combined_df_cluster1)

           mean expression  expressed cells (%)     cluster
HCRTR2            0.719916            99.348880  EN-ET-SP-A
NR4A2             1.420112            95.502347  EN-ET-SP-A
MLIP              0.602727            76.952539  EN-ET-SP-A
CYP26A1           0.673407            76.546547  EN-ET-SP-A
ST18              0.335150            51.879494  EN-ET-SP-A
STK32B            0.115034            23.180968  EN-ET-SP-A
UNC5C             0.099511            20.335737  EN-ET-SP-A
SLC26A4           0.034879             8.053096  EN-ET-SP-A
ADRA2A            0.666940            74.267080  EN-ET-SP-A
NPR3              0.318118            48.576838  EN-ET-SP-A
SATB2             1.078556            89.094013  EN-ET-SP-A
GABRA5            0.324419            51.031396  EN-ET-SP-A
CCN2              0.184081            29.275232  EN-ET-SP-A
COL19A1           0.080726            16.796708  EN-ET-SP-A
NR2F1             0.216569            39.175540  EN-ET-SP-A
KCNJ6             0.161767            30

In [293]:
combined_df_cluster1.to_csv("combined_df_cluster1.csv")
combined_df_cluster2.to_csv("combined_df_cluster2.csv")